# OC-IA-P10 - CHATBOT

# Training LUIS

Documentation V3 authoring:  
https://westeurope.dev.cognitive.microsoft.com/docs/services/luis-programmatic-apis-v3-0-preview/operations/5890b47c39e2bb052c5b9c45/console  

Documentation V3 prediction:  
https://westus.dev.cognitive.microsoft.com/docs/services/luis-endpoint-api-v3-0/operations/5cb0a91e54c9db63d589f433

In [2]:
import json
import pandas as pd
from dotenv import dotenv_values
from types import SimpleNamespace
from chatbot_app.luis_tools.luis_manager import LuisManager
from chatbot_app.entities_and_intents import entities, intents

params = SimpleNamespace(**dotenv_values("chatbot_app/.env"))
lm = LuisManager(
    params.luis_subscription_id,
    params.luis_app_id, 
    params.luis_version_id, 
    params.luis_authoring_key, 
    params.luis_authoring_endpoint, 
    params.luis_prediction_key,
    params.luis_prediction_endpoint,
    params.luis_slot_name,
)

In [ ]:
for intent in intents:
    lm.create_intent(intent)

In [ ]:
for entity in entities:
    lm.create_entity(entity)

In [4]:
for file in [
    'train_agree.json', 
    'train_disagree.json', 
    'train_greeting.json', 
    'train_inform.json']:

    with open(file) as f:
        data = json.load(f)
    response = lm.upload_samples(data)



INFO:root:Starting samples upload...
INFO:root:samples loaded.
INFO:root:Starting samples upload...
INFO:root:samples loaded.
INFO:root:Starting samples upload...
INFO:root:samples loaded.
INFO:root:Starting samples upload...
INFO:root:samples loaded.


In [5]:
lm.train_model()
training_status = lm.get_training_status()

INFO:root:Starting model training...
INFO:root:Training launched.
INFO:root:Model trained.


In [6]:
lm.get_prediction('I want to go to Montreal from Paris with a budget of 2000$')

INFO:root:Prediction: {'topIntent': 'inform', 'intents': {'inform': {'score': 0.83778095}}, 'entities': {'dst_city': ['Montreal'], 'or_city': ['Paris'], 'budget': ['2000$']}}


<Response [200]>

In [9]:
with open('../OC-IA-P10/test_data.json') as f:
    test_data = json.load(f)
test_data_upload_info = lm.upload_test_data(test_data)
operation_id = test_data_upload_info.json()['operationId']
operation_id

INFO:root:Test data uploaded.


'0fb9d28d-2b87-4a5a-95d0-7c5f0c2cbc62_637956864000000000'

In [11]:

test_status = lm.get_test_status(operation_id)
test_status.json()

{'operationId': '0fb9d28d-2b87-4a5a-95d0-7c5f0c2cbc62_637956864000000000',
 'status': 'succeeded',
 'createdDateTime': '08/10/2022 16:31:35',
 'lastActionDateTime': '08/10/2022 16:31:36'}

In [12]:
test_result = lm.get_test_result(operation_id)


In [13]:
for key in test_result.json().keys():
    display(pd.DataFrame(test_result.json()[key]))

,modelName,modelType,precision,recall,fScore
0,agree,Intent Classifier,1.00,0.58,0.74
1,disagree,Intent Classifier,0.80,0.31,0.44
2,greeting,Intent Classifier,0.69,0.78,0.74
3,inform,Intent Classifier,0.93,1.0,0.96
4,None,Intent Classifier,0.00,NaN,NaN


,modelName,modelType,precision,recall,fScore
0,budget,Entity Extractor,0.50,0.57,0.53
1,dst_city,Entity Extractor,0.50,0.55,0.52
2,end_date,Entity Extractor,0.56,0.67,0.61
3,or_city,Entity Extractor,0.45,0.51,0.48
4,str_date,Entity Extractor,0.42,0.53,0.47


,text,predictedIntentName,labeledIntentName,falsePositiveEntities,falseNegativeEntities
0,I'm looking for a trip to Gotham City leaving ...,inform,inform,"[{'entityName': 'budget', 'startCharIndex': 12...","[{'entityName': 'budget', 'startCharIndex': 12..."
1,"Hi, I need to go to Mos Eisley for a wedding, ...",inform,inform,"[{'entityName': 'dst_city', 'startCharIndex': ...","[{'entityName': 'dst_city', 'startCharIndex': ..."
2,"I'd like to get away from Monday, August 15, 2...",inform,inform,"[{'entityName': 'budget', 'startCharIndex': 94...","[{'entityName': 'budget', 'startCharIndex': 94..."
3,"Hello, I want to leave from Seoul on August 27...",inform,inform,"[{'entityName': 'or_city', 'startCharIndex': 2...","[{'entityName': 'or_city', 'startCharIndex': 2..."
4,I would like a vacation for one in Mannheim fr...,inform,inform,"[{'entityName': 'dst_city', 'startCharIndex': ...","[{'entityName': 'or_city', 'startCharIndex': 9..."
...,...,...,...,...,...
115,You got it wrong,greeting,disagree,[],[]
116,That sucks,greeting,disagree,[],[]
117,I want to chat with a real person please,inform,disagree,[],[]
118,Damn machine!,greeting,disagree,[],[]


In [14]:
response = lm.get_prediction('I want to go to Montreal from Paris with a budget of 2000$')

INFO:root:Prediction: {'topIntent': 'inform', 'intents': {'inform': {'score': 0.83778095}}, 'entities': {'dst_city': ['Montreal'], 'or_city': ['Paris'], 'budget': ['2000$']}}


In [15]:
import requests
data = {"versionId": lm.version_id,
                "isStaging": False, 
                "directVersionPublish": False
                }
response = requests.post(
    url=lm.request_publish_url, 
    headers=lm.authoring_headers,
    json=data)
print(response.status_code)
response.request.body

201


b'{"versionId": "0.1", "isStaging": false, "directVersionPublish": false}'

In [16]:
publishing = lm.publish_model()

INFO:root:Model published.
